In [1]:
import glob
import tensorflow as tf
from keras.models import Sequential, load_model
import numpy as np
import pandas as pd
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import matplotlib.pyplot as plt
import keras as k

Using TensorFlow backend.


In [2]:
data = pd.read_csv("stroke.csv", index_col=0)

In [3]:
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,
9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [4]:
data = data.drop("ever_married" , axis=1)

In [5]:
data.head()

,gender,age,hypertension,heart_disease,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,
9046,Male,67.0,0,1,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61.0,0,0,Self-employed,Rural,202.21,NaN,never smoked,1
31112,Male,80.0,0,1,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49.0,0,0,Private,Urban,171.23,34.4,smokes,1
1665,Female,79.0,1,0,Self-employed,Rural,174.12,24.0,never smoked,1


In [6]:
column_req = ["gender", "age", "hypertension", "work_type", "Residence_type", "avg_glucose_level", "bmi", "smoking_status","stroke"]

In [7]:
for column in data.columns:
    if data[column].dtype == np.number:
        continue
    data[column] = LabelEncoder().fit_transform( data[column] )

In [8]:
data.head()

,gender,age,hypertension,heart_disease,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,
9046,1,67.0,0,1,2,1,228.69,36.6,1,1
51676,0,61.0,0,0,3,0,202.21,NaN,2,1
31112,1,80.0,0,1,2,0,105.92,32.5,2,1
60182,0,49.0,0,0,2,1,171.23,34.4,3,1
1665,0,79.0,1,0,3,0,174.12,24.0,2,1


In [9]:
data = data.fillna(data.mean())
data.head()

,gender,age,hypertension,heart_disease,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,
9046,1,67.0,0,1,2,1,228.69,36.600000,1,1
51676,0,61.0,0,0,3,0,202.21,28.893237,2,1
31112,1,80.0,0,1,2,0,105.92,32.500000,2,1
60182,0,49.0,0,0,2,1,171.23,34.400000,3,1
1665,0,79.0,1,0,3,0,174.12,24.000000,2,1


In [10]:
X = data.drop(["stroke"], axis=1)
Y = data["stroke"]

In [11]:
X.head()

,gender,age,hypertension,heart_disease,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
id,,,,,,,,,
9046,1,67.0,0,1,2,1,228.69,36.600000,1
51676,0,61.0,0,0,3,0,202.21,28.893237,2
31112,1,80.0,0,1,2,0,105.92,32.500000,2
60182,0,49.0,0,0,2,1,171.23,34.400000,3
1665,0,79.0,1,0,3,0,174.12,24.000000,2


In [12]:
Y.head()


id
9046     1
51676    1
31112    1
60182    1
1665     1
Name: stroke, dtype: int64

In [13]:
x_scaler = MinMaxScaler()
x_scaler.fit(X)
column_names = X.columns
X[column_names]=x_scaler.transform(X)

In [14]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y, test_size= 0.2, shuffle = True)

In [15]:
print("Before OverSampling, counts of label '1': {}".format(sum(Y_train == 1))) 
print("Before OverSampling, counts of label '0': {} \n".format(sum(Y_train == 0))) 

Before OverSampling, counts of label '1': 198
Before OverSampling, counts of label '0': 3890 



In [16]:
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state = 2) 
X_train_res, Y_train_res = sm.fit_sample(X_train, Y_train.ravel()) 


In [17]:
print("After OverSampling, counts of label '1': {}".format(sum(Y_train_res == 1))) 
print("After OverSampling, counts of label '0': {}".format(sum(Y_train_res == 0))) 

Y_test.head()

After OverSampling, counts of label '1': 3890
After OverSampling, counts of label '0': 3890


id
31590    0
40548    0
52034    0
14189    0
21292    0
Name: stroke, dtype: int64

# Build model

In [18]:
model = k.Sequential()

In [19]:
model.add(k.layers.Dense(10,activation=tf.nn.relu,input_shape=(X_train_res.shape[1],)))
model.add(k.layers.Dense(64,activation=tf.nn.relu))
model.add(k.layers.Dense(32,activation=tf.nn.relu))
model.add(k.layers.Dense(32,activation=tf.nn.relu))
model.add(k.layers.Dense(16,activation=tf.nn.relu))
model.add(k.layers.Dense(1,activation =tf.nn.sigmoid))

# Compiling the model

In [20]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train the model

In [ ]:
history = model.fit(X_train_res, Y_train_res, epochs = 2000, batch_size = X_train_res.shape[0])

score, acc = model.evaluate(X_test, Y_test,
                            batch_size=X_test.shape[0])
print('Test score:', score)
print('Test accuracy:', acc)

Epoch 1/2000
7780/7780 [==============================] - 0s 57us/step - loss: 0.6930 - accuracy: 0.5143
Epoch 2/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.6919 - accuracy: 0.5716
Epoch 3/2000
7780/7780 [==============================] - 0s 4us/step - loss: 0.6909 - accuracy: 0.5938
Epoch 4/2000
7780/7780 [==============================] - 0s 5us/step - loss: 0.6901 - accuracy: 0.5969
Epoch 5/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.6893 - accuracy: 0.6087
Epoch 6/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.6885 - accuracy: 0.6244
Epoch 7/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.6877 - accuracy: 0.6343
Epoch 8/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.6869 - accuracy: 0.6413
Epoch 9/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.6861 - accuracy: 0.6451
Epoch 10/2000
7780/7780 [==============================] - 0s 2

7780/7780 [==============================] - 0s 2us/step - loss: 0.4867 - accuracy: 0.7738
Epoch 80/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.4846 - accuracy: 0.7742
Epoch 81/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.4826 - accuracy: 0.7749
Epoch 82/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.4806 - accuracy: 0.7758
Epoch 83/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.4788 - accuracy: 0.7754
Epoch 84/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.4770 - accuracy: 0.7767
Epoch 85/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.4753 - accuracy: 0.7780
Epoch 86/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.4736 - accuracy: 0.7796
Epoch 87/2000
7780/7780 [==============================] - 0s 4us/step - loss: 0.4720 - accuracy: 0.7803
Epoch 88/2000
7780/7780 [==============================] - 0s 3us/ste

Epoch 157/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.4298 - accuracy: 0.8042
Epoch 158/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.4293 - accuracy: 0.8045
Epoch 159/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.4290 - accuracy: 0.8054
Epoch 160/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.4286 - accuracy: 0.8053
Epoch 161/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.4282 - accuracy: 0.8053
Epoch 162/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.4278 - accuracy: 0.8057
Epoch 163/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.4274 - accuracy: 0.8060
Epoch 164/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.4269 - accuracy: 0.8066
Epoch 165/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.4265 - accuracy: 0.8071
Epoch 166/2000
7780/7780 [====================

Epoch 235/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.3968 - accuracy: 0.8222
Epoch 236/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.3964 - accuracy: 0.8254
Epoch 237/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.3959 - accuracy: 0.8238
Epoch 238/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.3955 - accuracy: 0.8251
Epoch 239/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.3951 - accuracy: 0.8258
Epoch 240/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.3948 - accuracy: 0.8249
Epoch 241/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.3946 - accuracy: 0.8266
Epoch 242/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.3943 - accuracy: 0.8252
Epoch 243/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.3940 - accuracy: 0.8283
Epoch 244/2000
7780/7780 [====================

Epoch 313/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.3707 - accuracy: 0.8458
Epoch 314/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.3703 - accuracy: 0.8465
Epoch 315/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.3702 - accuracy: 0.8460
Epoch 316/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.3701 - accuracy: 0.8454
Epoch 317/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.3696 - accuracy: 0.8460
Epoch 318/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.3692 - accuracy: 0.8473
Epoch 319/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.3689 - accuracy: 0.8468
Epoch 320/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.3688 - accuracy: 0.8470
Epoch 321/2000
7780/7780 [==============================] - 0s 2us/step - loss: 0.3685 - accuracy: 0.8454
Epoch 322/2000
7780/7780 [====================

...

Epoch 1400/2000
7780/7780 [==============================] - 0s 4us/step - loss: 0.2333 - accuracy: 0.9122
Epoch 1401/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2331 - accuracy: 0.9138
Epoch 1402/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2345 - accuracy: 0.9113
Epoch 1403/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2329 - accuracy: 0.9136
Epoch 1404/2000
7780/7780 [==============================] - 0s 4us/step - loss: 0.2329 - accuracy: 0.9125
Epoch 1405/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2315 - accuracy: 0.9135
Epoch 1406/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2306 - accuracy: 0.9140
Epoch 1407/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2303 - accuracy: 0.9145
Epoch 1408/2000
7780/7780 [==============================] - 0s 4us/step - loss: 0.2302 - accuracy: 0.9148
Epoch 1409/2000
7780/7780 [==========

Epoch 1477/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2283 - accuracy: 0.9154
Epoch 1478/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2300 - accuracy: 0.9136
Epoch 1479/2000
7780/7780 [==============================] - 0s 4us/step - loss: 0.2303 - accuracy: 0.9152
Epoch 1480/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2320 - accuracy: 0.9117
Epoch 1481/2000
7780/7780 [==============================] - 0s 4us/step - loss: 0.2299 - accuracy: 0.9143
Epoch 1482/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2294 - accuracy: 0.9143
Epoch 1483/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2280 - accuracy: 0.9159
Epoch 1484/2000
7780/7780 [==============================] - 0s 4us/step - loss: 0.2269 - accuracy: 0.9157
Epoch 1485/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2270 - accuracy: 0.9158
Epoch 1486/2000
7780/7780 [==========

Epoch 1554/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2255 - accuracy: 0.9152
Epoch 1555/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2249 - accuracy: 0.9166
Epoch 1556/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2244 - accuracy: 0.9166
Epoch 1557/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2239 - accuracy: 0.9174
Epoch 1558/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2235 - accuracy: 0.9184
Epoch 1559/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2237 - accuracy: 0.9172
Epoch 1560/2000
7780/7780 [==============================] - 0s 4us/step - loss: 0.2241 - accuracy: 0.9170
Epoch 1561/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2248 - accuracy: 0.9166
Epoch 1562/2000
7780/7780 [==============================] - 0s 4us/step - loss: 0.2258 - accuracy: 0.9154
Epoch 1563/2000
7780/7780 [==========

Epoch 1631/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2210 - accuracy: 0.9181
Epoch 1632/2000
7780/7780 [==============================] - 0s 4us/step - loss: 0.2214 - accuracy: 0.9181
Epoch 1633/2000
7780/7780 [==============================] - 0s 4us/step - loss: 0.2218 - accuracy: 0.9174
Epoch 1634/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2223 - accuracy: 0.9175
Epoch 1635/2000
7780/7780 [==============================] - 0s 4us/step - loss: 0.2239 - accuracy: 0.9159
Epoch 1636/2000
7780/7780 [==============================] - 0s 4us/step - loss: 0.2236 - accuracy: 0.9189
Epoch 1637/2000
7780/7780 [==============================] - 0s 4us/step - loss: 0.2242 - accuracy: 0.9154
Epoch 1638/2000
7780/7780 [==============================] - 0s 4us/step - loss: 0.2226 - accuracy: 0.9179
Epoch 1639/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2222 - accuracy: 0.9171
Epoch 1640/2000
7780/7780 [==========

Epoch 1708/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2184 - accuracy: 0.9189
Epoch 1709/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2181 - accuracy: 0.9199
Epoch 1710/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2181 - accuracy: 0.9201
Epoch 1711/2000
7780/7780 [==============================] - 0s 4us/step - loss: 0.2180 - accuracy: 0.9194
Epoch 1712/2000
7780/7780 [==============================] - 0s 4us/step - loss: 0.2184 - accuracy: 0.9202
Epoch 1713/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2186 - accuracy: 0.9184
Epoch 1714/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2193 - accuracy: 0.9188
Epoch 1715/2000
7780/7780 [==============================] - 0s 3us/step - loss: 0.2213 - accuracy: 0.9161
Epoch 1716/2000
7780/7780 [==============================] - 0s 4us/step - loss: 0.2215 - accuracy: 0.9190
Epoch 1717/2000
7780/7780 [==========

In [42]:
model.save('ckd.model')

In [43]:
print("shape of train data", X_train_res.shape)
print("shape of test data", X_test.shape)

shape of train data (7796, 9)
shape of test data (1022, 9)


Cell for user input